In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import os
import seaborn as sns
from scipy import stats
from madule.simulation import *

In [2]:
# np.random.seed(123)

### Generate True Parameters For each Participant on Hierarchical level

In [4]:
"""Parameter recovery assessment.
True parameters for each participant are taken from hierarchical level parameters"""

# Number of simulation
simNumber = 1

# Set mean and std of unkown parameters
alphaAct_mu = np.round(np.random.rand(2, 2), 2)
alphaAct_sd = np.round(np.random.rand(2, 2), 2)

alphaClr_mu = np.round(np.random.rand(2, 2), 2)
alphaClr_sd = np.round(np.random.rand(2, 2), 2)

weghtAct_mu = np.round(np.random.rand(2, 2), 2)
weghtAct_sd = np.round(np.random.rand(2, 2), 2)

beta_mu = 5*np.round(np.random.rand(2, 2), 2)
beta_sd = 5*np.round(np.random.rand(2, 2), 2)

# call trueParam function to generate and save true parameters for each participant
trueParam(alphaAct_mu=alphaAct_mu, alphaAct_sd=alphaAct_sd,
          alphaClr_mu=alphaClr_mu, alphaClr_sd=alphaClr_sd,
          weghtAct_mu=weghtAct_mu, weghtAct_sd=weghtAct_sd,
          beta_mu=beta_mu, beta_sd=beta_sd,
          simNumber=simNumber)

## Simulation RL Data

In [3]:
def softmax(values, beta = 1):
    """Choices are selected by the soft-max function rule in each trial"""
    nom = np.exp(values[0]*beta)
    denom = np.sum([nom + np.exp(values[1]*beta)])
    return nom/denom

def sample_Bernouli(theta = .5, n_samples = 1):
    """
    Generating samples at random from Bernouli density funtion
    """
    return (np.random.rand(n_samples) <= theta).astype(int) 
    
def simulateActClr(task_design_param): 

    # Number of trials
    n_trials = task_design_param.shape[0]
    
    # condition
    block = task_design_param.block.to_numpy()
    
    # which action and color are available options
    winAmtPushable = task_design_param.winAmtPushable.to_numpy()
    winAmtPullable = task_design_param.winAmtPullable.to_numpy()
    winAmtYellow = task_design_param.winAmtYellow.to_numpy()
    winAmtBlue = task_design_param.winAmtBlue.to_numpy()  
    
    # available options on left and right side
    leftCanBePushed = task_design_param.leftCanBePushed.to_numpy()
    yellowOnLeftSide = task_design_param.yellowOnLeftSide.to_numpy()
    
    # Correct responces
    pushCorrect = task_design_param.pushCorrect.to_numpy()
    yellowCorrect = task_design_param.yellowCorrect.to_numpy()
    
    # True Unkown Parameters
    alphaAct = task_design_param.alphaAct.to_numpy()
    alphaClr = task_design_param.alphaClr.to_numpy()
    weghtAct = task_design_param.weghtAct.to_numpy()
    beta = task_design_param.beta.to_numpy()
    
    # output of simulation
    correctChoice = np.zeros(n_trials).astype(int)
    pushed = np.zeros(n_trials).astype(int)
    yellowChosen = np.zeros(n_trials).astype(int)


    # Initial expected probability
    probPush = .5
    probPull = .5
    probYell = .5
    probBlue = .5
    
    for i in range(n_trials):
        # Standard Expected Value 
        expValuePush = probPush*winAmtYellow[i]
        expValuePull = probPull*winAmtBlue[i]
        expValueYell = probBlue*winAmtPushable[i]
        expValueBlue = probBlue*winAmtPullable[i]

        # Relative contribution of Action Value Learning verus Color Value Learning
        expValuePushYell = weghtAct[i]*expValuePush + (1 - weghtAct[i])*expValueYell;
        expValuePushBlue = weghtAct[i]*expValuePush + (1 - weghtAct[i])*expValueBlue;
        expValuePullYell = weghtAct[i]*expValuePull + (1 - weghtAct[i])*expValueYell;
        expValuePullBlue = weghtAct[i]*expValuePull + (1 - weghtAct[i])*expValueBlue;

        # Calculating the soft-max function over weightening Action and Color conditions*/ 
        if (leftCanBePushed[i] == 1 and yellowOnLeftSide[i] == 1) and (leftCanBePushed[i] == 0 and yellowOnLeftSide[i] == 0):
            """pushed and yellow vs pulled and blue"""
            theta = softmax(values=[expValuePushYell, expValuePullBlue], beta=beta[i])
            
            # make a response
            y = sample_Bernouli(theta = theta) 
            
            # Response for the current trial
            if y==1:
                pushed[i] = 1
                yellowChosen[i] = 1
            else:
                pushed[i] = 0
                yellowChosen[i] = 0
                
        elif (leftCanBePushed[i] == 1 and yellowOnLeftSide[i] == 0) or (leftCanBePushed[i] == 0 and yellowOnLeftSide[i] == 1):
            """pushed and blue vs pulled and yellow"""
            theta = softmax(values=[expValuePushBlue, expValuePullYell], beta=beta) 
           
            # make a response
            y = sample_Bernouli(theta = theta)        
            
            # Response for the current trial
            if y==1:
                pushed[i] = 1
                yellowChosen[i] = 0
            else:
                pushed[i] = 0
                yellowChosen[i] = 1
                
        # Get reward
        if block[i] == 'Act':
            reward = int(pushed[i] == pushCorrect[i])
            # Choice correct for the current trial
            correctChoice[i] =  reward
        elif block[i] == 'Stim':
            reward = int(yellowChosen[i] == yellowCorrect[i])
            # Choice correct for the current trial
            correctChoice[i] =  reward
        
        # Rl rule update
        if pushed[i] == 1:
            probPush = probPush + alphaAct[i]*(reward - probPush)
            probPull = 1 - probPush           
        else:
            probPull = probPull + alphaAct[i]*(reward - probPull)
            probPush = 1 - probPull                      
        if yellowChosen[i] == 1:
            probYell = probYell + alphaClr[i]*(reward - probYell)
            probBlue = 1 - probYell
        else:
            probBlue = probBlue + alphaClr[i]*(reward - probBlue)
            probYell = 1 - probBlue  
        
    # output results
    task_design['correctChoice'] = correctChoice
    task_design['pushed'] = pushed
    task_design['yellowChosen'] = yellowChosen 
    
    # Put true parameter into task design dataframe
    task_design['alphaAct'] = alphaAct 
    task_design['alphaClr'] = alphaClr 
    task_design['weghtAct'] = weghtAct 
    task_design['beta'] = beta  
    
    return task_design